<a href="https://colab.research.google.com/github/nihermann/Pokemaenner/blob/main/Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
##################################### Console #####################################
# !git clone https://github.com/nihermann/Pokemaenner.git
%cd Pokemaenner/
# !git status
###################################################################################

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#@title # Using Gan to create new Pokemon
from datetime import datetime
import tensorflow as tf
import utils
from data import DataGenerator
from aegan import (AEGAN, SaveAegan)

In [10]:
# @title # Model
model = None
# @markdown ## AEGAN
use_aegan = True  # @param {type:"boolean"}
if use_aegan:
    image_shape = (64, 64, 3)  # @param
    latentspace = 24  # @param {type:"integer"}
    batch_size = 32  # @param {type:"integer"}
    noise_generating_function = lambda b: tf.random.normal((b, latentspace))  # @param {type:"raw"}
    continue_from_saved_models = False  # @param {type:"boolean"}
    only_weights = True  # @param {type:"boolean"}
    path = "/content/drive/MyDrive/pokemaenner//outputs/models"  # @param ["./models"] {allow-input: true}

    batch_size *= 8

    model = AEGAN(
        image_shape=image_shape,
        latentspace=latentspace,
        batch_size=batch_size,
        noise_generating_fn=noise_generating_function,
        continue_from_saved_models=continue_from_saved_models,
        only_weights=only_weights,
        path=path
    )

Build models: [====]
Build successfully


In [11]:
# @title ## Data Settings

image_path = "/content/drive/MyDrive/pokemaenner/data64/"  # @param {type:"string"}
images_in_test_split = 20  # @param {type:"slider", min:4, max:20, step:4}
horizontal_flip = False  # @param {type:"boolean"}
shuffle = True  # @param {type:"boolean"}

data = DataGenerator(
    img_path=image_path,
    batch_size=batch_size,
    img_size=image_shape[:2],
    images_in_test_split=images_in_test_split,
    horizontal_flip=horizontal_flip,
    shuffle=shuffle
)

Found 26113 images belonging to 1 classes.
Found 20 images belonging to 1 classes.


In [ ]:
# @title # Training Parameters
epochs = 200  # @param {type:"integer"}
samples_per_epoch = 100  # @param {type:"integer"}
print_verbose = "progressbar"  # @param ["no_prints", "print_after_each_epoch", "progressbar"]
print_verbose = {"no_prints": 0, "print_after_each_epoch": 2, "progressbar": 1}[print_verbose]

# @markdown ## Callbacks
# @markdown ### Model saving
callbacks = []
# save_models = False  # @param {type:"boolean"}
# if save_models:
#     model_path = "./outputs/models/aegan{epoch:03d}.h5"  # @param ["./outputs/models/"] {allow-input: true}
#
#     callbacks.append(
#         tf.keras.callbacks.ModelCheckpoint(
#             filepath=model_path,
#             save_weights_only=save_weights_only,
#         )
#     )

# @markdown ### Tensorboard
use_tensorboard = False  # @param {type:"boolean"}
log_dir = None
timestemp = datetime.now().strftime("%Y%m%d_%H%M%S")
if use_tensorboard:
    log_dir = "./logs/images"  # @param ["./logs"] {allow-input: true}
    update_frequency = "epoch"  # @param ["batch", "epoch"] {allow-input: true}

    log_dir += ('' if log_dir.endswith('/') else '/') + timestemp

    print(f"TensorBoard logdir: {log_dir}")

    callbacks.append(
        tf.keras.callbacks.TensorBoard(
            log_dir=log_dir,
            update_freq=update_frequency
        )
    )


    def launchTensorBoard():
        import os
        os.system('tensorboard --logdir=' + log_dir)
        return


    import threading

    t = threading.Thread(target=launchTensorBoard, args=([]))
    t.start()

# @markdown ### Save Pictures and Weights for AEGAN
# @markdown Images will be saved to file and displayed in tensorboard if activated.
# @markdown The number of images equals to the amount of pictures specified in the validation data split.
save_aegan = True  # @param {type:"boolean"}
if save_aegan:
    pictures_path = "/content/drive/MyDrive/pokemaenner/outputs/"  # @param ["./output/"] {allow-input: true}
    save_pictures_every =   2# @param {type:"integer"}
    save_model_every = 10  # @param {type:"integer"}
    save_only_weights = False  # @param {type:"boolean"}

    callbacks.append(
        SaveAegan(
            save_images_every=save_pictures_every,
            save_model_every=save_model_every,
            only_weights=save_only_weights,
            save_path=pictures_path,
            data_gen=data.validation_generator,
            tensorboard_logdir=log_dir
        )
    )

# @markdown ### Save History to csv
# @markdown The name will be generated automatically so you only need to specify the path where it should be saved.
save_history = True  # @param {type:"boolean"}
if save_history:
    history_path = "/content/drive/MyDrive/pokemaenner/outputs/history"  # @param ["./outputs/history"] {allow-input: true}
    history_path = utils.setup_path(history_path)
    callbacks.append(
        tf.keras.callbacks.CSVLogger(
            history_path + f"aegan_64_b{batch_size//8}_l{latentspace}_s{samples_per_epoch}_history_log_{timestemp}.csv",
            append=True
        )
    )

assert model is not None, "Model must not be None, please make sure to enable one of them by setting the respective " \
                          "bool to True!"
print("Start training..")
model.fit(
    x=data.training_generator,
    steps_per_epoch=samples_per_epoch,
    epochs=epochs + model.initial_epoch,
    verbose=print_verbose,
    callbacks=callbacks,
    initial_epoch=model.initial_epoch
)